# Practice Detection

Detect routines, habits, and rituals from mobility patterns.

## Practice Types
- **Routine**: Regular pattern (e.g., daily commute)
- **Habit**: Automatic behavior at a place
- **Ritual**: Meaningful practice with symbolic significance

In [ ]:
from chora.core import PlatialGraph, Agent, SpatialExtent, Encounter, PlatialEdge
from datetime import datetime, timedelta

graph = PlatialGraph(name="practice_demo")
agent = Agent.individual("commuter")
graph.add_node(agent)

## Create Regular Patterns

In [ ]:
# Create locations
home = SpatialExtent.from_point(-0.09, 51.51, name="Home")
office = SpatialExtent.from_point(-0.10, 51.52, name="Office")
coffee_shop = SpatialExtent.from_point(-0.095, 51.515, name="Coffee Shop")
gym = SpatialExtent.from_point(-0.085, 51.508, name="Gym")

for ext in [home, office, coffee_shop, gym]:
    graph.add_node(ext)

encounters = []

In [ ]:
# Simulate 3 weeks of weekday patterns
base_date = datetime.now() - timedelta(weeks=3)

for week in range(3):
    for day in range(5):  # Mon-Fri
        current_date = base_date + timedelta(weeks=week, days=day)
        
        # Morning: Home -> Coffee Shop -> Office (daily routine)
        for hour, place in [(7, home), (7.5, coffee_shop), (9, office)]:
            enc = Encounter(
                agent_id=agent.id,
                extent_id=place.id,
                start_time=current_date.replace(hour=int(hour), minute=int((hour % 1) * 60)),
                activity="commute" if place != coffee_shop else "coffee"
            )
            graph.add_node(enc)
            graph.add_edge(PlatialEdge.participates_in(agent.id, enc.id))
            graph.add_edge(PlatialEdge.occurs_at(enc.id, place.id))
            encounters.append(enc)
        
        # Evening gym (Mon, Wed, Fri only)
        if day in [0, 2, 4]:
            enc = Encounter(
                agent_id=agent.id,
                extent_id=gym.id,
                start_time=current_date.replace(hour=18),
                activity="exercise"
            )
            graph.add_node(enc)
            graph.add_edge(PlatialEdge.participates_in(agent.id, enc.id))
            graph.add_edge(PlatialEdge.occurs_at(enc.id, gym.id))
            encounters.append(enc)

print(f"Created {len(encounters)} encounters over 3 weeks")

## Detect Practices

In [ ]:
from chora.derive.practices import detect_practices, PracticeDetectionConfig

config = PracticeDetectionConfig(
    min_occurrences=3,
    time_window_hours=2.0,
    regularity_threshold=0.5
)

practices = detect_practices(encounters, str(agent.id), config)

print(f"Detected {len(practices)} practices:\n")
for p in practices:
    print(f"{p.name}")
    print(f"  Type: {p.practice_type.name}")
    print(f"  Regularity: {p.regularity:.2f}")
    print(f"  Frequency: {p.frequency:.1f} times/week")
    print()

## Analyze Practice Patterns

In [ ]:
from chora.core.types import PracticeType

# Group by type
by_type = {}
for p in practices:
    ptype = p.practice_type.name
    if ptype not in by_type:
        by_type[ptype] = []
    by_type[ptype].append(p)

print("Practices by type:")
for ptype, items in by_type.items():
    print(f"  {ptype}: {len(items)} practices")

## Detect Sequences

In [ ]:
from chora.derive.practices import find_sequence_patterns

sequences = find_sequence_patterns(encounters, min_support=0.3)

print("Detected sequence patterns:")
for seq in sequences:
    places = " → ".join(seq.locations)
    print(f"  {places} (support: {seq.support:.2f})")